In [4]:
import numpy as np
import xgboost as xgb

xgb_dir = "/home/yannick/github/xgboost/demo"

### load data in do training
dtrain = xgb.DMatrix(xgb_dir + '/data/agaricus.txt.train')
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2

print ('running cross validation')
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value+std_value
# std_value is standard deviation of the metric
xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'error'}, seed = 0)

print ('running cross validation, disable standard deviation display')
# do cross validation, this will print result out as
# [iteration]  metric_name:mean_value+std_value
# std_value is standard deviation of the metric
xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'error'}, seed = 0, show_stdv = False)

print ('running cross validation, with preprocessing function')
# define the preprocessing function
# used to return the preprocessed training, test data, and parameter
# we can use this to do weight rescale, etc.
# as a example, we try to set scale_pos_weight
def fpreproc(dtrain, dtest, param):
    label = dtrain.get_label()
    ratio = float(np.sum(label == 0)) / np.sum(label==1)
    param['scale_pos_weight'] = ratio
    return (dtrain, dtest, param)

# do cross validation, for each fold
# the dtrain, dtest, param will be passed into fpreproc
# then the return value of fpreproc will be used to generate
# results of that fold
xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'auc'}, seed = 0, fpreproc = fpreproc)

###
# you can also do cross validation with cutomized loss function
# See custom_objective.py
##
print ('running cross validation, with cutomsized loss function')
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0-preds)
    return grad, hess
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

param = {'max_depth':2, 'eta':1, 'silent':1}
# train with customized objective
xgb.cv(param, dtrain, num_round, nfold = 5, seed = 0,
       obj = logregobj, feval=evalerror)


running cross validation
running cross validation, disable standard deviation display
running cross validation, with preprocessing function
running cross validation, with cutomsized loss function


,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.055760,0.015827,0.050691,0.009194
1,0.021198,0.003813,0.021313,0.002075


In [13]:
r = xgb.cv(param, dtrain, num_round, nfold=5,
       metrics={'error'}, seed = 0)
print(r)
print(r.tail(1)["train-error-mean"])

   test-error-mean  test-error-std  train-error-mean  train-error-std
0         0.055760        0.015827          0.050691         0.009194
1         0.022273        0.015384          0.019316         0.009833
1    0.019316
Name: train-error-mean, dtype: float64
